# Programación en Paralelo

* ### Distribuir funciones de cálculo entre distintos procesadores
* ### Colectar los resultados obtenidos en cada procesador
* ### Sintetizar con esos resultados parciales, un resultado significativo.

## Pase de mensajes entre procesadores

## M P I = Message Passing Interface

* ### Es una especificación, no es una librería de funciones.
* ### Muchas organizaciones se ponen de acuerdo
* ### Diseñada para implementar modelos de programación en paralelo
* ### Los datos se mueven de un espacio de memoria de un proceso al espacio de memoria de otro proceso
* ### Con MPI se pueden escribir aplicaciones que requieren pasar mensajes entre ellos
* ### La especificación MPI tiene varias versiones, hay que tener cuidado con esto


## Modelo de programación

* ### Originalmente diseñada para arquitecturas de memoria distribuída
![Memoria Distribuída](distributed_mem.gif)

* ### Evolución arquitectural  SMP (Symmetric multiprocessing) con memoria compartida
![Sistemas  híbridos](hybrid_mem.gif)

* ### MPI soporta modelos:
  * Distribuidos
  * Compartidos
  * Híbridos
  
* ### El modelo de programación es en memoria distribuída, independiente de la arquitectura física
* ### El paralelismo es explícito: el programador identifica el paralelismo y es el responsable de implementar el paralelismo


## Sistemas de memoria distribuída

* ### Cada procesador tiene su propia memoria
* ### Una red de datos conecta todo los procesadores

## Programación por pase de mensajes

* ### Un programa en paralelo se descompone en procesadores, llamados ranks
* ### Cada rank mantiene una parte de los datos de programas en su memoria
* ### La comunicación entre ranks se explicita con mensajes entre ellos
* ### Los mensajes responden a FIFO

## Un solo programa, múltiples datos

* ### Todos los procesadores corren el mismo programa
* ### Todos los programas empiezan simultaneamente
* ### Comunicaciones entre nodos
  * #### Mensajes punto a punto
  * #### Operaciones de comunicación colectiva (ej. broadcast)
  
  
## Principales características del pase de mensaje

* ### Simple: operaciones de comunicación típicas (Send, Receive)
* ### General: puede ser implementado en la mayoría de las arquitecturas paralelo}
* ### Performante: una implementación se puede adaptar a un hardware particular
* ### Escalable: el mismo programa se puede usar en sistemas más grandes
  

## MPI Ranks

* ### Tienen su propia memoria
* ### Cada  rank tiene un único número de identificación
* ### Los ranks se numeran secuencialmente: [0, N-1].

![mpi ranks](mpi_ranks.png)

## Comunicadores MPI



* ### Grupo de ranks entre los cuales un nodo del grupo se puede comunicar
* ### COMM_WORLD es un comunicador que contiene a todos los ranks (nodos) del sistema

![comm world](comm_world.png)

## Razones para usar MPI

* ### Standard
* ### Portable
* ### Optimizable
* ### Funcional
* ### Disponible (C, Fortran, python, c++, otros ..)
* ### Lenguaje standard de facto para cálculo en paralelo en computadoras alta performance
* ### Es un modelo simple de comunicaciones entre procesos

## Estructura de Programa

![Sistemas  híbridos](prog_structure.gif)

## Python: mpi4py

* ### Una de las mejores implementaciones de MPI en python
* ### Cumple con la especificación MPI-2 (hay MPI-3)

### HelloWorld.py

In [ ]:
"""
Parallel Hello World
"""
#
# Los includes
#
from mpi4py import MPI
 
#
# Inicializacion
#
size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
name = MPI.Get_processor_name()

#
# El programa
#
print(
    "Hello, World! I am process %d of %d on %s."
    % (rank, size, name))

### Ejecución


    > mpiexec -n 4  python helloworld.py
    
### Salida

    Hello, World! I am process 2 of 4 on sheldon.dcap.cnea.gov.ar.
    Hello, World! I am process 3 of 4 on sheldon.dcap.cnea.gov.ar.
    Hello, World! I am process 1 of 4 on sheldon.dcap.cnea.gov.ar.
    Hello, World! I am process 0 of 4 on sheldon.dcap.cnea.gov.ar.

### baseParallel.py

In [ ]:
#!/usr/bin/env python
"""
Parallel baseParallel 
"""

#
# Los include
#
from mpi4py import MPI

#
# La inicializacion
#
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#
# Definicion de la funcion
#
def func(rank):
    print ("func ejecuto en %d" % (rank))

#
# Ejecucion de la funcion
#
func(rank)

### Ejecución


    > mpiexec -n 4  python baseParallel.py

### Salida

    func ejecuto en 1
    func ejecuto en 2
    func ejecuto en 0
    func ejecuto en 3

### parallelSum.py

In [ ]:
#!/usr/bin/env python
"""
Parallel parallelSum
"""

from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#
# Los datos para operar
#
a = [1,2,4,5]
b = [10,20,40,50]


def func(rank):
    c = a[rank] + b[rank]
    print ("ejecuto en %d %d" % (rank, c))


func(rank)

### Ejecución

    mpiexec -n 4 python parallelSum.py

### Salida
    ejecuto en 3 55
    ejecuto en 0 11
    ejecuto en 1 22
    ejecuto en 2 44
    
### Observar el orden de ejecución de los nodos    

### parallelSum_B.py

In [ ]:
#!/usr/bin/env python
"""
Parallel parallelSum
"""

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = [1,2,4,5]
b = [10,20,40,50]

def func(rank):
    c = a[rank] + b[rank]
    print ("ejecuto en %d %d" % (rank, c))
    return c


S = func(rank)

print( "nodo %d suma %d"%(rank, S))

### Ejecución

    mpiexec -n 4 python parallelSum_B.py
    
### Salida

    ejecuto en 2 44
    nodo 2 suma 44
    ejecuto en 3 55
    nodo 3 suma 55
    ejecuto en 0 11
    nodo 0 suma 11
    ejecuto en 1 22
    nodo 1 suma 22

### parallelSum_C.py

In [ ]:
#!/usr/bin/env python
"""
Parallel parallelSum
"""

from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = [1,2,4,5]
b = [10,20,40,50]
total = 0

def func(rank):
    c = a[rank] + b[rank]
    print ("ejecuto en %d %d" % (rank, c))
    return c


S = func(rank)

if rank == 0:
    total = total + S
else:
    pass



print ("total: ", total)

### Ejecución

    mpiexec -n 4 python  parallelSum_C.py

### Salida

    ejecuto en 0 11
    ('total: ', 11)
    ejecuto en 1 22
    ('total: ', 0)
    ejecuto en 2 44
    ('total: ', 0)
    ejecuto en 3 55
    ('total: ', 0)


### parallelSum_D.py

In [ ]:
#!/usr/bin/env python
"""
Parallel parallelSum
"""

from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = [1,2,4,5]
b = [10,20,40,50]
total = 0

def func(rank):
    c = a[rank] + b[rank]
    print ("ejecuto en %d %d" % (rank, c))
    return c


S = func(rank)

if rank == 0:
    total = total + S
else:
    pass


if rank == 0:
    print ("total: ", total)

### Ejecución    
    
    mpiexec -n 4 python  parallelSum_D.py


### Salida
    ejecuto en 2 44
    ejecuto en 3 55
    ejecuto en 0 11
    ('total: ', 11)
    ejecuto en 1 22


### parallelSum_E.py

In [ ]:
#!/usr/bin/env python
"""
Parallel parallelSum_E
"""

#
#   Las librerias que vamos a usar
#
from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE
import numpy as np

#
# Inicio de MPI
#
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#
# Los datos del problema
#
a = np.asarray([1,2,4,5])
b = np.asarray([10,20,40,50])
total = 0

#
# La memoria para transmitir y recibir resultados
#
suma = np.zeros(1)
recv_buffer = np.zeros(1)

#
# La funcion a ejecutar 
#
def func(rank):
    c = a[rank] + b[rank]
    print ("ejecuto en %d %d" % (rank, c))
    return c

#
# la ejecucion de la funcion
#
suma[0] = func(rank)


#
# Concentracion de los resultados
#
if rank == 0:
    total = total + suma[0]
    # el nodo 0 espera los resultados
    for i in range(1, size):
        comm.Recv(recv_buffer, ANY_SOURCE)
        total += recv_buffer[0]
else:
    # el resto de los nodos transmiten el resultado S 
    comm.Send(suma)


if rank == 0:
        print ("total: ", total)

### Ejecución

    mpiexec -n 4 python  parallelSum_E.py
    
#### Salida

    ejecuto en 0 11
    ejecuto en 2 44
    ejecuto en 3 55
    ejecuto en 1 22
    ('total: ', 132.0)

## Referencias

* Message Passing Interface  [(MPI)](https://computing.llnl.gov/tutorials/mpi/)
* Serial to Parallel: Monte Carlo Operation [link](https://www.olcf.ornl.gov/tutorials/monte-carlo-pi/)

* A Python Introduction to Parallel Programming with MPI [link](http://materials.jeremybejarano.com/MPIwithPython/index.html)

* MPI with MPI4py Introduction [link](http://pythonprogramming.net/learning-use-mpi-python-mpi4py-module/)